In [44]:
import os

import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import pandas as pd
from ODEDataset import ODEDataset
from typing import Callable, Any
# FunctionType = Callable[[int, int], str]

# Appliances Data
- Presence_indoor_light_bulb
- m_indoor_light_bulb
- P_indoor_light_bulb
- Presence_outdoor_light_bulb
- m_outdoor_light_bulb
- P_outdoor_light_bulb
- Presence_light_bulb
- m_light_bulb
- Presence_radio
- m_radio
- P_radio
- Presence_stereo
- m_stereo
- P_stereo
- Presence_radio/stereo
- m_radio/stereo


In [45]:
ds = ODEDataset("FUNAE/Chissinguana").from_excel("../playground/data/FUNAE/Chissinguana/Chissinguana.xlsx")

df = ds.to_dataframe()

df.head()


,ID,Respondent_category,Type_of_business,Type_of_public_service,Street_lightling_presence,Presence_light_bulb_simple_list,m_light_bulb_simple_list,P_light_bulb_simple_list,Presence_security_light_list,m_security_light_list,...,P_printer,Presence_computer,m_computer,P_computer,Presence_sewing_machine,m_sewing_machine,P_sewing_machine,Presence_hair_dryer,m_hair_dryer,P_hair_dryer
0,Household_load_1_01,Household,NaN,NaN,Yes,1,5,20,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Household_load_1_02,Household,NaN,NaN,Yes,1,5,20,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Household_load_1_03,Household,NaN,NaN,Yes,1,5,20,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Household_load_1_04,Household,NaN,NaN,Yes,1,5,20,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Household_load_1_05,Household,NaN,NaN,Yes,1,5,20,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:

def extract_ready_appliance_and_rename(cols: dict) -> callable:
    def inner_extractor(src: pd.DataFrame) -> pd.DataFrame:
        df = src.copy()
        # print(cols, df.columns, cols.keys())

        df = df[cols.keys()]
        df.head()
        df.rename(columns=cols, inplace=True)
        return df

    return inner_extractor

In [47]:
ds = ds.apply(
    extract_ready_appliance_and_rename(
        {
            # indoor
            "Presence_light_bulb_simple_list": "Presence_indoor_light_bulb",
            "m_light_bulb_simple_list": "m_indoor_light_bulb",
            "P_light_bulb_simple_list": "P_indoor_light_bulb",
            # outdoor light
            "Presence_security_light_list": "Presence_outdoor_light_bulb",
            "m_security_light_list": "m_outdoor_light_bulb",
            "P_security_light_list": "P_outdoor_light_bulb",
            #radio
            "Presence_radio_list": "Presence_radio",
            "m_radio_list": "m_radio",
            "P_radio_list": "P_radio",
            #stereo
            "Presence_stereo_list": "Presence_stereo",
            "m_stereo_list": "m_stereo",
            "P_stereo_list": "P_stereo",

        }
    )
)
ds.to_dataframe()

,Presence_indoor_light_bulb,m_indoor_light_bulb,P_indoor_light_bulb,Presence_outdoor_light_bulb,m_outdoor_light_bulb,P_outdoor_light_bulb,Presence_radio,m_radio,P_radio,Presence_stereo,m_stereo,P_stereo
0,1,5,20,0,0,0,0,0,0,0,0,0
1,1,5,20,0,0,0,0,0,0,0,0,0
2,1,5,20,0,0,0,0,0,0,0,0,0
3,1,5,20,0,0,0,0,0,0,0,0,0
4,1,5,20,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
60,1,2,20,0,0,0,0,0,0,0,0,0
61,1,12,30,0,0,0,0,0,0,0,0,0
62,1,2,20,0,0,0,0,0,0,0,0,0
63,1,2,20,0,0,0,0,0,0,0,0,0


# Drivers
- Survey_date
- GADM_level_0
- Urban/rural
- System_management
- Population
- Connection_type
- Township/Village
- Latitude
- Longitude
- Distance_from_grid
- Distance_from_city
- GADM_level_1
- GADM_level_2
- GADM_level_3
- Climate_zone_lev_1
- Climate_zone_lev_2
- Climate_zone_lev_3
- Population_density_lev_2
- Population_density_lev_3
- Relative_wealth_index_lev_2
- Relative_wealth_index_lev_3
- masl_lev_2
- masl_lev_3


In [48]:
def add_const_driver(name: str, value) -> callable:
    def inner_modifier(src: pd.DataFrame) -> pd.DataFrame:
        df = src.copy()
        df[name] = value
        return df

    return inner_modifier


def add_const_driver_many(driver: [dict]) -> callable:
    def inner_modifier(src: pd.DataFrame) -> pd.DataFrame:
        df = src.copy()
        for name, value in driver.items():
            df[name] = value

        return df

    return inner_modifier

In [49]:
ds = ds.apply(add_const_driver_many({
    "Survey_date": 2017,
    "GADM_level_0": "Rural",
    "Urban/rural": "rural",
    "System_management": "Community",
    "Population": 25622,
    "Connection_type": "Local mini-grid",
    "Township/Village": "Chissinguane",
}))

ds.to_dataframe().head()

,Presence_indoor_light_bulb,m_indoor_light_bulb,P_indoor_light_bulb,Presence_outdoor_light_bulb,m_outdoor_light_bulb,P_outdoor_light_bulb,Presence_radio,m_radio,P_radio,Presence_stereo,m_stereo,P_stereo,Survey_date,GADM_level_0,Urban/rural,System_management,Population,Connection_type,Township/Village
0,1,5,20,0,0,0,0,0,0,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane
1,1,5,20,0,0,0,0,0,0,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane
2,1,5,20,0,0,0,0,0,0,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane
3,1,5,20,0,0,0,0,0,0,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane
4,1,5,20,0,0,0,0,0,0,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane


In [71]:
def combine_drivers(result: str, func: Callable[[pd.Series], Any]) -> callable:
    def inner_modifier(src: pd.DataFrame) -> pd.DataFrame:
        df = src.copy()
        df[result] = df.apply(func, axis=1)
        return df

    return inner_modifier


def unify_presence(driver1: str, driver2: str) -> Callable:
    def inner_unify_presence(row: pd.Series) -> int:
        x = row[driver1]
        y = row[driver2]
        if x == 1 or y == 1:
            return 1
        else    :
            return 0

    return inner_unify_presence


def sum_presence(driver1: str, driver2: str) -> Callable:
    def inner_sum_presence(row: pd.Series) -> int:
        x = row[driver1]
        y = row[driver2]
        return x + y

    return inner_sum_presence



In [72]:
def remove_drivers(drivers: [str]) -> callable:
    def inner_modifier(src: pd.DataFrame) -> pd.DataFrame:
        df = src.copy()
        return df.drop(columns=drivers)

    return inner_modifier

In [74]:
# indoor and outdoor light


ds = (ds.apply(combine_drivers("Presence_light_bulb",unify_presence("Presence_indoor_light_bulb", "Presence_outdoor_light_bulb")))
    .apply(remove_drivers(["Presence_indoor_light_bulb", "Presence_outdoor_light_bulb"]))
       )


ds = (ds.apply(combine_drivers("m_light_bulb",sum_presence("m_indoor_light_bulb", "m_outdoor_light_bulb")))
       .apply(remove_drivers(["m_indoor_light_bulb", "m_outdoor_light_bulb"]))
       )



# radio and stereo
ds = (ds.apply(combine_drivers("Presence_radio/stereo",unify_presence("Presence_radio", "Presence_stereo")))
    .apply(remove_drivers(["Presence_radio", "Presence_stereo"]))
         )

ds = (ds.apply(combine_drivers("m_radio/stereo",sum_presence("m_radio", "m_stereo")))
         .apply(remove_drivers(["m_radio", "m_stereo"]))
         )



In [75]:
df = ds.to_dataframe()
df

,P_indoor_light_bulb,P_outdoor_light_bulb,P_radio,P_stereo,Survey_date,GADM_level_0,Urban/rural,System_management,Population,Connection_type,Township/Village,Presence_light_bulb,m_light_bulb,Presence_radio/stereo,m_radio/stereo
0,20,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane,1,5,0,0
1,20,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane,1,5,0,0
2,20,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane,1,5,0,0
3,20,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane,1,5,0,0
4,20,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane,1,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,20,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane,1,2,0,0
61,30,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane,1,12,0,0
62,20,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane,1,2,0,0
63,20,0,0,0,2017,Rural,rural,Community,25622,Local mini-grid,Chissinguane,1,2,0,0
